# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [11]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2022-11-19	"Preserving the Privacy
and Cybersecurity of Home Energy Data"	Emerging Trends in Cybersecurity Applications	This paper is about the number 1. The number 2 is left for future work.	"Richard Bean, Yanjun Zhang, Ryan KL Ko, Xinyu Mao and Guangdong Bai. 2023. Preserving the Privacy
and Cybersecurity of Home Energy Data. Emerging Trends in Cybersecurity Applications. Springer Publishing."	"Preserving the Privacy
and Cybersecurity of Home Energy Data"	https://pure.rug.nl/ws/portalfiles/portal/563519192/978_3_031_09640_2.pdf#page=328	http://academicpages.github.io/files/slides3.pdf
2022-12-15	Robustness of Neural Rankers to Typos: A Comparative Study	Proceedings of the 26th Australasian Document Computing Symposium	This paper is about the number 2. The number 3 is left for future work.	"Shengyao Zhuang, Xinyu Mao and Guido Zuccon. 2022. Robustness of Neural Rankers to Typos: A Comparative Study. In Proceedings of 

## Import pandas

We are using the very handy pandas library for dataframes.

In [3]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [18]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2022-11-19,Preserving the Privacy\nand Cybersecurity of H...,Emerging Trends in Cybersecurity Applications,This paper is about the number 1. The number 2...,"Richard Bean, Yanjun Zhang, Ryan KL Ko, Xinyu ...",Privacy-home-energy,https://pure.rug.nl/ws/portalfiles/portal/5635...,http://academicpages.github.io/files/slides3.pdf
1,2022-12-15,Robustness of Neural Rankers to Typos: A Compa...,Proceedings of the 26th Australasian Document ...,This paper is about the number 2. The number 3...,"Shengyao Zhuang, Xinyu Mao and Guido Zuccon. 2...",neural-ranker-typos,https://ielab.io/publications/adcs2022-typos/a...,http://academicpages.github.io/files/slides3.pdf
2,2024-03-15,A Reproducibility Study of Goldilocks: Just-Ri...,Proceedings of the 46th European Conference on...,This paper is about the number 3. The number 4...,"Xinyu Mao, Bevan Koopman and Guido Zuccon. 202...",goldilocks-reproduce,http://academicpages.github.io/files/paper3.pdf,http://academicpages.github.io/files/slides3.pdf
3,2024-07-10,Dense Retrieval with Continuous Explicit Feedb...,Proceedings of the 47th International ACM SIGI...,This paper is about the number 1. The number 2...,"Xinyu Mao, Shengyao Zhuang, Bevan Koopman and ...",dense-feedback,https://arxiv.org/pdf/2407.00635,http://academicpages.github.io/files/slides3.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [19]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [20]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [21]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md
2010-10-01-paper-title-number-2.md
2015-10-01-paper-title-number-3.md
2022-11-19-Preserving the Privacy?and Cybersecurity of Home Energy Data.md
2022-11-19-Privacy-home-energy.md
2022-12-15-neural-ranker-typos.md
2024-02-17-paper-title-number-4.md
2024-03-15-goldilocks-reproduce.md
2024-07-10-dense-feedback.md


In [22]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
slidesurl: 'http://academicpages.github.io/files/slides1.pdf'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---

The contents above will be part of a list of publications, if the user clicks the link for the publication than the contents of section will be rendered as a full page, allowing you to provide more information about the paper for the reader. When publications are displayed as a single page, the contents of the above "citation" field will automatically be included below this section in a smaller font.